In [1]:
from numpy import *
import numpy as np
import pandas as pd
import lxml.html as lh
from lxml.html import parse
import urllib.request, json
from urllib.request import urlopen
from lxml import objectify
import urllib
from zipfile import ZipFile
from bs4 import BeautifulSoup
import requests, zipfile, io
import html5lib

from fuzzywuzzy import fuzz
from fuzzywuzzy import process


C:\Users\kredfield\AppData\Local\Continuum\Anaconda3\lib\site-packages\fuzzywuzzy\fuzz.py:35: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
def variable_standardize(df):
    #function to standardize columns upon readin
    df.rename(columns=dict(zip([x for x in df.columns],[x.lstrip() for x in df.columns])), inplace=True)
    df.rename(columns=dict(zip([x for x in df.columns],[x.rstrip() for x in df.columns])), inplace=True)
    df.rename(columns=dict(zip([x for x in df.columns],[x.replace(' ', '') for x in df.columns])), inplace=True)
    df.rename(columns=dict(zip([x for x in df.columns],[x.replace('#', 'num') for x in df.columns])), inplace=True)
    df.rename(columns=dict(zip([x for x in df.columns],[x.replace('/', '_') for x in df.columns])), inplace=True)
    df.rename(columns=dict(zip([x for x in df.columns],[x.replace(',', '') for x in df.columns])), inplace=True)
    df.rename(columns=dict(zip([x for x in df.columns],[x.replace('.', '') for x in df.columns])), inplace=True)
    df.columns = map(str.lower, df.columns)

    return df
    
def drop_duplicate_columns(df):
    #function to identify and then drop duplicated column names
    
    #get col list
    Cols = list(df.columns)
    
    #start looping through
    for i,item in enumerate(df.columns):
        #change the name of the column to "toDROP" if its already occured
        if item in df.columns[:i]: Cols[i] = "toDROP"
    df.columns = Cols
    #drop the identified duplicates
    df = df.drop("toDROP",1)
    
    return df

In [3]:
url = "http://www.areep.com/rwt/results/results_master.php"

#open up the url and turn it into an html response element
read_text = urlopen(url).read()

#extract all tables from the html
table = lh.fromstring(read_text)

# extract the text from `<td>` tags
data = [[elt.text_content() for elt in tr.xpath('td')] for tr in table.xpath('//tr')]

#and turn the text into dataframes 
#use the first row of the table as the header
df = pd.DataFrame(data[2:], columns=data[1])

#make a blank dataframe with the columns you want to fill for appending later
dfcolumns = ['place', 'bib', 'name', 'age', 'sex', 'div', 'city', 'time', 'nettime','pace', 'total_participants']
race_results = pd.DataFrame(columns = dfcolumns)
big_sheet = pd.DataFrame(columns = dfcolumns)

# extract the href attribute values into a new column
df['refname'] = table.xpath('//tr/td/a/@href')

#start looking into the compiled dataframe for specific race results
for i, row in df.iterrows():
    url = df.ix[i,'refname']
    
    #a little bit of user control here in case the program craps out and you need to restart it from a point further on
    #set threshold to 0 if you want to start from the beginning
    if i > 0:
        
        #anything that isn't in the format of "zippyraceresults.com" will return errors, so we'll just look at these
        if "zippyraceresults" in url:
            try:
                race_table = pd.read_html(url, parse_dates = True)
                
            except ValueError:
                print(url, "has no tables")
                continue
            except TypeError:
                print(url, "has type error")
                print(i)
                continue
            except:
                print(url)
                print(i)
                raise
            
            #the way the table is returned, you only need every other row
            #so only take the first out of every set of two rows
            race_table = race_table[0].iloc[::2].reset_index(drop=True)
            
            #standardize the variable names using the function above
            race_table = variable_standardize(race_table)
            
            #test if there are duplicate columns and, if there are, drop the duplicates
            if len(race_table.columns) != len(set(race_table.columns)):
                race_table = drop_duplicate_columns(race_table)
            
            #count up the total number of people running in the race
            race_table['total_participants'] = race_table.count()[0]

            #sometimes, the table won't include the place
            #if that's true, we'll take matters into our own hands
            if "place" not in race_table.columns:
                    race_table.sort_values("time", inplace=True)
                    race_table["place"] = race_table.rank()
                    
            #only keep the top 10
            try:
                race_table = race_table[race_table["place"]<=50]
                race_table = race_table.dropna(axis=1, how="all")

            except KeyError:
                print(url, "is missing place still")
                race_table.sort_values("time")
                continue
            except TypeError:
                print(url, "has type error")
                print(i)
                continue
            except:
                print(url, "other error type")
                print(i)
                raise
        else: 
            print(url)
            continue
            
        #in your returned result to the big sheet, capture the name of the race and the date
        race_table["race_name"] = row.Name
        race_table['date'] = row.Date
        
        #put this all together to get a full record of the race and the top 10
        race_results = pd.concat([race_table,race_results],ignore_index=True)
        
        #and stick it on the running tab of results
        big_sheet = pd.concat([big_sheet,race_results],ignore_index=True)
        
        #clear the definition for race_results
        del race_results
        race_results = pd.DataFrame(columns = dfcolumns)
        continue

http://www.dcroadrunners.org/races/race-results/2017-results/3160-2017-age-handicapped-4-miler.html
https://www.athlinks.com/event/20899/results/Event/603815/Results
https://www.athlinks.com/event/33857/results/Event/612117/Results
http://racepacket.com/rsltwrap1.php?id=7721
https://www.athlinks.com/Master/138877/Event/609930/Results
http://www.runrocknroll.com/finisher-zone/search-and-results/?eventid=42
http://www.racepacket.com/tidalbasin/20170215.html
https://www.athlinks.com/Master/13536/Event/586956/Course/882049/Results
https://results.chronotrack.com/event/results/event/event-22712
http://www.athletic.net/CrossCountry/Results/Meet.aspx?Meet=116060
http://results.xacte.com/?mid=613
https://results.chronotrack.com/event/results/event/event-25231
https://www.athletic.net/CrossCountry/Results/Meet.aspx?Meet=118783
https://results.chronotrack.com/event/results/event/event-25075
https://results.chronotrack.com/event/results/event/event-19723
https://s3.amazonaws.com/media.racebx.com/

IncompleteRead: IncompleteRead(7568960 bytes read, 8517765 more expected)

In [ ]:
#work off a copied version
all_races = big_sheet.copy()

#Combine names if they are separated into first and last
all_races.name = np.where(all_races.name.isnull(),all_races.firstname + " " + all_races.lastname, all_races.name)
#if the field "time" is populated instead of "nettime", fill over
all_races.nettime = np.where(all_races.nettime.isnull(),all_races.time, all_races.nettime)

#fill state with last two chars of city if blank
all_races["StateClean"] = np.where(all_races.state.isnull(),all_races.city.str[(all_races.city.str.len()-2):],all_races.state)

#uppercase state
all_races["StateClean"] = all_races.StateClean.str.upper()


#delete last two char of state if city is blank
all_races["CityClean"] = np.where(all_races.state.isnull(),all_races.city.str[:(all_races.city.str.len()-3)],all_races.city)
#propercase city
all_races.CityClean = all_races.CityClean.str.title()



#Set up the columns you want to keep in the raw file
keep_cols = ['age','date','CityClean','name','nettime','pace','place','race_name','sex','StateClean','total_participants']
drop_list = []

for col in all_races.columns:
    if col not in keep_cols:
        #put columns that you don't want into a list to drop later
        drop_list.append(col)
    else:
        
        #but, if you want to keep them, strip leading and trailing
        if isinstance(all_races.ix[1,col],str):
            all_races[col] = all_races[col].str.strip()

#drop those columns you put in a list to drop
all_races.drop(drop_list, axis=1, inplace=True)

#set up all the appropriate types of races in the file
race_types = ['5K','10K','8K','10 Miler','Marathon','Half Marathon','3.5 Miler','20K','Four Miler', "15K", '9K', '5 Miler', 'Metric Marathon' ,'50 Miler', '9.2K', '3 Miler', '4.5 Miler', '3K', '5000m', "Pi Miler", "1 Miler", "2 Miler", "4 Miler", "50K"]

#and apply them in a field called race_type
all_races['race_type'] = all_races.race_name.apply(lambda x: process.extractOne(x, race_types)[0] if process.extractOne(x, race_types)[1] >= 60 else "No Match")
check_type = all_races[all_races.race_type == "No Match"]



#send cleaned file to excel

writer = pd.ExcelWriter(r'C:\Users\kredfield\Documents\Washington Run Times\All_Races.xlsx')
big_sheet.to_excel(writer, index=False, sheet_name="raw")
all_races.to_excel(writer, index=False, sheet_name="Top50")
writer.save()

all_races